In [45]:
import lightgbm as lgb
from sklearn.metrics import f1_score, roc_auc_score

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 6,
    'min_data_in_leaf': 1100,
    'feature_fraction': 0.5,
    'bagging_fraction': 1,
    'verbose': 0
}

In [4]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/"
train = pd.read_csv(data+'train_1.csv')
valid = pd.read_csv(data+'valid_1.csv')

In [20]:
def training(train, valid):
    X = lgb.Dataset(train.drop('rating', axis=1), train['rating'])
    V = lgb.Dataset(valid.drop('rating', axis=1), valid['rating'], reference=X)
    gbdt = lgb.train(params, X, valid_sets=[X,V], early_stopping_rounds=20, verbose_eval=False)
    Yvp = gbdt.predict(valid.drop('rating', axis=1), num_iteration=gbdt.best_iteration)
    Ytp = gbdt.predict(train.drop('rating', axis=1), num_iteration=gbdt.best_iteration)
    return (roc_auc_score(train['rating'].values, Ytp), roc_auc_score(valid['rating'].values, Yvp))

In [28]:
def cv(param, paramlst):
    trainauc = [0.0]*len(paramlst)
    validauc = [0.0]*len(paramlst)
    for i, p in enumerate(paramlst):
        params[param]=p
        tv = [0,0,0]
        vv = [0,0,0]
        for fold in [1,2,3]:
            t = pd.read_csv(data+'train_{0}.csv'.format(str(fold)))
            v = pd.read_csv(data+'valid_{0}.csv'.format(str(fold)))
            tv[fold-1], vv[fold-1] = training(t, v)
        trainauc[i]=np.mean(tv)
        validauc[i]=np.mean(vv)
    paramtable = pd.DataFrame({
        'TrainingSet': trainauc,
        'ValidationSet': validauc
    }, columns=['TrainingSet', 'ValidationSet'], index=pd.Index(paramlst, name=param))
    print(paramtable)

In [49]:
cv('min_gain_to_split', [0.1, 1, 10, 100])

                   TrainingSet  ValidationSet
min_gain_to_split                            
0.1                   0.917199       0.799041
1.0                   0.917199       0.799041
10.0                  0.917199       0.799041
100.0                 0.914841       0.795885


In [16]:
[]*8

[]